In [2]:
import os
import pandas as pd
import numpy as np
from scipy import misc
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
np.random.seed(31415)

In [3]:
img_path = "/root/sharedfolder/clean/"
legend = pd.read_csv("/root/sharedfolder/legend/legend-clean.csv")

In [4]:
batch_size = 128
nb_classes = 8
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 96, 96
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [5]:
legend['emotion'] = legend['emotion'].str.lower()

In [6]:
# 'neutral' = 0
# 'happiness' = 1
# 'surprise' = 2
# 'anger' = 3
# 'sadness' = 4
# 'contempt' = 5
# 'disgust' = 6
# 'fear' = 7
legend.replace('neutral', 0, inplace=True)
legend.replace('happiness', 1, inplace=True)
legend.replace('surprise', 2, inplace=True)
legend.replace('anger', 3, inplace=True)
legend.replace('sadness', 4, inplace=True)
legend.replace('contempt', 5, inplace=True)
legend.replace('disgust', 6, inplace=True)
legend.replace('fear', 7, inplace=True)

In [7]:
# convert images to numpy.ndarray

images = np.empty(shape=[len(legend), img_rows, img_cols])
for i in range(0,len(legend)):
    temp = img_path + legend.iloc[i,0]
    img = misc.imread(temp, flatten=True,)
    img = misc.imresize(img, (img_rows, img_cols))
    images[i] = img

In [8]:
emotions = legend['emotion']

In [9]:
# the data, shuffled and split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, emotions, test_size=0.25)

In [11]:
X_train.shape

(10260, 96, 96)

In [12]:
X_test.shape

(3421, 96, 96)

In [13]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

In [14]:
X_train.shape

(10260, 1, 96, 96)

In [15]:
X_test.shape

(3421, 1, 96, 96)

In [16]:
input_shape = (1, img_rows, img_cols)

In [17]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('X_train shape:', (10260, 1, 96, 96))
(10260, 'train samples')
(3421, 'test samples')
Train on 10260 samples, validate on 3421 samples
Epoch 1/12
10260/10260 [==============================] - 627s - loss: 1.3992 - acc: 0.5146 - val_loss: 1.0344 - val_acc: 0.4426
Epoch 2/12
10260/10260 [==============================] - 705s - loss: 0.9767 - acc: 0.6138 - val_loss: 0.8331 - val_acc: 0.7325
Epoch 3/12
10260/10260 [==============================] - 715s - loss: 0.8508 - acc: 0.6969 - val_loss: 0.7797 - val_acc: 0.7241
Epoch 4/12
10260/10260 [==============================] - 716s - loss: 0.7557 - acc: 0.7383 - val_loss: 0.7759 - val_acc: 0.7469
Epoch 5/12
10260/10260 [==============================] - 740s - loss: 0.7005 - acc: 0.7544 - val_loss: 0.6970 - val_acc: 0.7814
Epoch 6/12
10260/10260 [==============================] - 712s - loss: 0.6570 - acc: 0.7696 - val_loss: 0.6275 - val_acc: 0.7837
Epoch 7/12
10260/10260 [==============================] - 720s - loss: 0.6133 - acc: 0.789